# Decision Tree Classifier from Scratch #2

## 1. Importing Libraries

In [2]:
from __future__ import print_function

## 2. Dataset

* Each row is an example
* The last column is the label
* The first two columns are features

In [3]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

#column Labels
header = ["color", "diameter", "label"]

## 3. HELPER FUNCTION: Finding the unique values for a column in the dataset

* unique_vals(rows, col)
    * rows: accepts a dataset
    * col: will be used to specify the column of the dataset
    * For the specified column of the dataset, it will return the unique values of that column


In [4]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

## 4. HELPER FUNCTION: Count the number of each type of example in the dataset

* class_counts(rows):
    * rows: accpets a dataset

In [5]:
def class_counts(rows):
    #Dictionary for counts
    counts = {}

    for row in rows:
        label = row[-1] #label is equal to the last column in the dataset
        if label not in counts:
            counts[label] = 0
        counts[label] += 1

    return counts

class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

## 5. HELPER FUNCTION: Testing if a value is numeric

In [6]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

is_numeric(7)

True

## 6. Partitioning the dataset

In [7]:
class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]

        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="

        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))
    
Question(1,3)

Is diameter >= 3?

## 7. Testing above cases with example

In [8]:
case1 = Question(1, 3) #The question is: Is diameter >= 3
case2 = Question(0, 'Green') #The question is: Is color == Green

example = training_data[0] # Example = ['Green', 3, 'Apple']  

print(case1.match(example))
print(case2.match(example))


True
True


## 8. Partitioning the dataset

For each row in the dataset, check if it matche the question.
* If it does, add it to 'True Rows' 
* Else, add it to 'False Rows'

In [9]:
def partition(rows, question):
    #Creating an empty list for the true and false rows
    true_rows, false_rows = [],[] 

    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)

    return true_rows, false_rows

#Example of partitioning the dataset based on whether rows are 'Red'
partition(training_data, Question(0, 'Red')) #returns the True Rows first, followed by the False Rows

([['Red', 1, 'Grape'], ['Red', 1, 'Grape']],
 [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']])

## 9. Calculating Gini Index

Gini Index Forumla: Forumal: 1−∑(*pi*)^2 



In [14]:
def gini_index(rows):
    counts = class_counts(rows) #Will return: {'Apple': 2, 'Grape': 2, 'Lemon': 1}
    impurity = 1

    for label in counts:
        #print("Counts Label: ", counts[label])
        probability_of_label = counts[label] / float(len(rows))
        #print(probability_of_label)
        impurity -= probability_of_label**2
        # print(impurity)

    return impurity

print("Gini Index(Impurity): ", gini_index(training_data))

Gini Index(Impurity):  0.6399999999999999


## 10. Calculating Information Gain
* The uncertainty of the starting node, minus the weigbhted impurity of two child nodes

In [15]:
def info_gain(left_child, right_child, current_uncertainty):
    p = float(len(left_child)) / (len(left_child) + len(right_child))

    return current_uncertainty - p * gini_index(left_child) - (1 - p) * gini_index(right_child)

#Calculating Information Gain below:
current_uncertainty = gini_index(training_data) #current_uncertainty = 0.639

true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# True Rows = [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
# False Rows = [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

#For this next step, True Rows are left_child, and False Rows are right_child
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

## 11. Finding the best split
Finding the best question to ask by iterating over every feature/value and calculating the information gain

In [18]:
def best_split(rows):
    
    #Keeps track of the best information gain
    best_gain = 0
    #Keeps track of the feature/value that produces the best_gain
    best_question = None
    #Find the current uncertainty for the rows in the dataset
    current_uncertainty = gini_index(rows)
    #Number of columns
    n_features = len(rows[0]) - 1

    for column in range(n_features): #For each feature
        #Unique values in the column
        values = set([row[column] for row in rows])

        for value in values: #For each value
            question = Question(column, value)

            #Splitting the dataset
            true_rows, false_rows = partition(rows, question)

            #Skipping if the split doesnt divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            #Calculate the Information Gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


best_gain, best_question = best_split(training_data)
best_question

Is diameter >= 3?

## 12. Leaf Class

In [19]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

## 13. Decision Node

In [20]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

## 14. Building the Tree

In [22]:
def build_tree(rows):

    # 1. Try partitioning the dataset on each of the unique features
    # 2. Calculate the information gain
    # 3. Return the quetion that produces the highest gain

    gain, question = best_split(rows)

    # Base case: no futher information gain. Return a leaf since we can ask no further questions
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature/value to partition on
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch
    true_branch = build_tree(true_rows)

    # Same for false branch
    false_branch = build_tree(false_rows)

    #Return a Question node
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing = ""):

    # Base case: we have reached a leaf
    if isinstance(node, Leaf):22
        print(spacing + "Predict", node.predictions)
        return
    
    #Print the question at this node
    print(spacing + str(node.question))

    #Call this function recurively on the true branch
    print(spacing + '--> True: ')
    print_tree(node.true_branch, spacing + " ")

    #Call this function recursively on the false branch
    print(spacing + '--> False: ')
    print_tree(node.false_branch, spacing + " ")

tree1 = build_tree(training_data)
print_tree(tree1)

Is diameter >= 3?
--> True: 
 Is color == Yellow?
 --> True: 
  Predict {'Apple': 1, 'Lemon': 1}
 --> False: 
  Predict {'Apple': 1}
--> False: 
 Predict {'Grape': 2}
